In [ ]:
import sys 
sys.path.append("../")

import nerf_model
import dataloader
import nerf_helpers

import cv2
from PIL import Image

import gc 
import torch 
import itertools 
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
network = nerf_model.NeRFNetwork(position_dim=10, direction_dim=4, coarse_samples=64,
                 fine_samples=128)
network.training_step(batch, 0)

In [ ]:
def visualize(coords, rgb): 
    a, b = coords.shape
    if b == 3:
        coords = coords.T
    if type(rgb) != str:
        rgb = rgb.T
    plot_fig = go.Scatter3d(x=coords[0], y=coords[1], z=coords[2], 
    mode='markers', marker=dict(
       size=2,
       color=rgb
    ))
    return plot_fig

In [ ]:
from importlib import reload
dataloader = reload(dataloader)
nerf_model = reload(nerf_model)
nerf_helpers = reload(nerf_helpers)
gc.collect()

In [ ]:
base_dir = '../data/lego/'
sdl = dataloader.getSyntheticDataloader(base_dir, 'val', 4096, num_workers=1, shuffle=True)

batch = next(iter(sdl))
nerf_helpers.fix_batchify(batch)
print(batch.keys())
print(batch['rgba'].shape)
print(batch['origin'].shape)
print(batch['direc'].shape)
print(batch['xs'].shape)
print(batch['all_origin'].shape)


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for batch in iter(sdl):
    gc.collect()

    origins = batch['all_origin'].view((-1,3))[::10,:]
    direcs = batch['all_direc'].view((-1, 3))[::10,:]

    points = origins + direcs

    pic = visualize(points, 'pink')
    o = visualize(origins, 'black')
    fig.add_trace(pic)
    fig.add_trace(o)
fig.show()